<a target="_blank" href="https://shiyanlou.com/louplus/ml"><img style="float: right;" src="https://doc.shiyanlou.com/document-uid214893labid7506timestamp1555400601684.png"></a>

## 构建 LeNet-5 Estimator

---

<i class="fa fa-exclamation-circle" aria-hidden="true"> 以下内容仅包含挑战需学员补充部分的参考答案，完整挑战请到原课程页面查看。</i>

---

In [ ]:
import tensorflow as tf


def lenet_fn(features, labels, mode):
    # 卷积层，6 个 5x5 卷积核，步长为 1，relu 激活，第一层需指定 input_shape
    conv1 = tf.keras.layers.Conv2D(filters=6,
                                   kernel_size=(5, 5),
                                   strides=(1, 1),
                                   activation='relu', input_shape=(32, 32, 1))(features["x"])
    # 平均池化，池化窗口默认为 2
    pool1 = tf.keras.layers.AveragePooling2D(
        pool_size=(2, 2), strides=2)(conv1)
    # 卷积层，16 个 5x5 卷积核，步为 1，relu 激活
    conv2 = tf.keras.layers.Conv2D(filters=16, kernel_size=(
        5, 5), strides=(1, 1), activation='relu')(pool1)
    # 平均池化，池化窗口默认为 2
    pool2 = tf.keras.layers.AveragePooling2D(
        pool_size=(2, 2), strides=2)(conv2)
    # 需展平后才能与全连接层相连
    flatten = tf.keras.layers.Flatten()(pool2)
    # 全连接层，输出为 120，relu 激活
    fc1 = tf.keras.layers.Dense(units=120, activation='relu')(flatten)
    # 全连接层，输出为 84，relu 激活
    fc2 = tf.keras.layers.Dense(units=84, activation='relu')(fc1)
    # 全连接层，输出为 10，Softmax 激活
    logits = tf.keras.layers.Dense(units=10, activation='softmax')(fc2)

    # 计算损失
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # 训练模式
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # 评估模式
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(
                labels=labels, predictions=tf.argmax(input=logits, axis=1))}
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
mnist_lenet = tf.estimator.Estimator(model_fn=lenet_fn)
mnist_lenet

In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train.astype('float32')},
    y=y_train.astype('int32'),
    batch_size=32,
    num_epochs=3,
    shuffle=True)

In [ ]:
mnist_lenet.train(input_fn=train_input_fn)

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test.astype('float32')},
    y=y_test.astype('int32'),
    batch_size=32,
    num_epochs=1,
    shuffle=False)

In [ ]:
mnist_lenet.evaluate(input_fn=eval_input_fn)

---

<img src="https://doc.shiyanlou.com/document-uid214893labid7506timestamp1545810029884.png">